In [1]:
!pip -q install faiss-cpu sentence-transformers requests tqdm


In [2]:
from pathlib import Path
import json
import numpy as np
import faiss
import requests
from sentence_transformers import SentenceTransformer
from typing import List, Dict

INDEX_FILE = Path("vector_db/iea_faiss.index")
META_FILE  = Path("vector_db/iea_metadata.jsonl")

assert INDEX_FILE.exists(), "FAISS index not found. Run vector_store.ipynb first."
assert META_FILE.exists(), "Metadata file not found. Run vector_store.ipynb first."


/Users/adityaamiddha/hello/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
index = faiss.read_index(str(INDEX_FILE))
print("✅ Loaded FAISS index | vectors:", index.ntotal)

metadata = []
with META_FILE.open("r", encoding="utf-8") as f:
    for line in f:
        metadata.append(json.loads(line))

print("✅ Loaded metadata records:", len(metadata))


✅ Loaded FAISS index | vectors: 352
✅ Loaded metadata records: 352


In [4]:
EMBED_MODEL = "BAAI/bge-small-en-v1.5"
embedder = SentenceTransformer(EMBED_MODEL)
print("✅ Loaded embedder:", EMBED_MODEL)


✅ Loaded embedder: BAAI/bge-small-en-v1.5


In [5]:
MODEL_NAME = "phi3:mini"

def ollama_generate(
    prompt: str,
    model: str = MODEL_NAME,
    max_tokens: int = 220,
    temperature: float = 0.2,
    top_p: float = 0.9,
    context_window: int = 2048,
    timeout_sec: int = 60
) -> str:
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {
            "num_predict": max_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "num_ctx": context_window
        }
    }
    r = requests.post(url, json=payload, timeout=timeout_sec)
    r.raise_for_status()
    return r.json().get("response", "").strip()


In [6]:
def retrieve(query: str, k: int = 5) -> List[Dict]:
    qv = embedder.encode(query, normalize_embeddings=True).astype("float32")
    D, I = index.search(np.array([qv]), k)

    results = []
    for score, idx in zip(D[0], I[0]):
        if idx == -1:
            continue
        item = metadata[idx].copy()
        item["score"] = float(score)
        results.append(item)
    return results


In [7]:
SYSTEM_PROMPT = """
You are VoltAI, a domain-specific assistant for electric vehicle market trends, charging infrastructure, battery ecosystem, and EV policy.
You must answer ONLY using the provided EVIDENCE from the IEA Global EV Outlook knowledge base.

Rules:
1) Use ONLY information present in EVIDENCE.
2) If the answer is not found in EVIDENCE, respond exactly with: "Insufficient data in knowledge base."
3) Be factual, concise, and avoid assumptions.
4) When relevant, include numbers and year references.
5) End with a Sources section listing sources used.
"""

def format_evidence(chunks: List[Dict], max_chars_per_chunk: int = 650) -> str:
    formatted = []
    for i, ch in enumerate(chunks, 1):
        txt = ch["text"].replace("\n", " ").strip()
        txt = txt[:max_chars_per_chunk]
        formatted.append(
            f"[EVIDENCE {i}] (SOURCE={ch['source']}, YEAR={ch['year']}, CHUNK_ID={ch['chunk_id']}, SCORE={ch.get('score',0):.4f})\n"
            f"{txt}"
        )
    return "\n\n".join(formatted)

def format_chat_history(chat_history: List[Dict], max_turns: int = 6) -> str:
    if not chat_history:
        return "None"
    recent = chat_history[-max_turns:]
    lines = []
    for msg in recent:
        role = msg["role"].upper()
        content = msg["content"].strip()
        lines.append(f"{role}: {content}")
    return "\n".join(lines)

def build_prompt(user_query: str, retrieved_chunks: List[Dict], chat_history: List[Dict] = None) -> str:
    evidence_block = format_evidence(retrieved_chunks)
    history_block = format_chat_history(chat_history or [])

    prompt = f"""
{SYSTEM_PROMPT}

CHAT HISTORY:
{history_block}

EVIDENCE:
{evidence_block}

USER QUESTION:
{user_query}

INSTRUCTIONS:
- Use bullet points.
- Keep answer under 140 words.
- No hallucinations.
- End with Sources: ...
"""
    return prompt.strip()


In [8]:
def voltai_answer(
    user_query: str,
    chat_history: List[Dict],
    top_k: int = 5
) -> Dict:
    """
    Returns:
    {
      "answer": str,
      "sources": list,
      "evidence": list of chunks (top-k),
      "chat_history": updated chat history
    }
    """

    # 1) Retrieve evidence
    retrieved = retrieve(user_query, k=top_k)

    # 2) Build prompt using evidence + memory
    prompt = build_prompt(user_query, retrieved, chat_history=chat_history)

    # 3) Generate response with Ollama
    answer = ollama_generate(prompt)

    # 4) Collect sources
    sources = []
    for ch in retrieved:
        src = {"source": ch["source"], "year": ch["year"], "chunk_id": ch["chunk_id"], "score": ch["score"]}
        sources.append(src)

    # 5) Update chat history
    chat_history = chat_history + [
        {"role": "user", "content": user_query},
        {"role": "assistant", "content": answer}
    ]

    return {
        "answer": answer,
        "sources": sources,
        "evidence": retrieved,
        "chat_history": chat_history
    }


In [9]:
chat_history = []

result = voltai_answer(
    user_query="What are the key global EV adoption trends discussed in 2023?",
    chat_history=chat_history,
    top_k=5
)

print("ANSWER:\n", result["answer"])
print("\nSOURCES:\n", result["sources"][:3])


ANSWER:
 - Increased government targets for EV adoption in major markets, including policy support for vehicle and battery manufacturing as well as critical mineral supply chains (EVIDENCE 1).

- Global spending on electric cars by governments and consumers significantly increased to USD 400 billion in 2022. Policy requirements were a key driver of electrification for companies, particularly within major markets such as China, Europe, and the United States (EVIDENCE 1).
- Electric car registrations remain concentrated with just under 60% occurring in China, about 25% in Europe, and around 10% in the U.S., indicating regional disparities in EV adoption rates (EVIDENCE 3).

Sources: IEA Global EV Outlook knowledge base - Evidence ID [IEA_2023_000034], [IEA_2025_000044], and

SOURCES:
 [{'source': 'GEVO2023_clean', 'year': 2023, 'chunk_id': 'IEA_2023_000034', 'score': 0.8367905616760254}, {'source': 'GlobalEVOutlook2025_clean', 'year': 2025, 'chunk_id': 'IEA_2025_000044', 'score': 0.82113

In [10]:
chat_history = []

r1 = voltai_answer("What does the report say about EV sales growth in 2023?", chat_history, top_k=5)
chat_history = r1["chat_history"]

r2 = voltai_answer("What about charging infrastructure growth?", chat_history, top_k=5)
chat_history = r2["chat_history"]

print("Turn 1 Answer:\n", r1["answer"][:800])
print("\nTurn 2 Answer:\n", r2["answer"][:800])


Turn 1 Answer:
 - In 2023, global electric vehicle (EV) sales are expected to reach nearly 14 million units, marking a significant increase of about 35% compared to the previous year and boosting EVs' market share to approximately 18%. This growth is supported by declining costs and strengthened policy support in key regions such as the United States.
- The report highlights that China has set an ambitious target for NEV sales, aiming for a 50% share by 2025. Xpeng Motors' expansion into national EV leadership is attributed to this goal and includes BEVs, PHEVs, and fuel cell electric vehicles in their portfolio.
- The United States saw first-quarter sales of around 350,000 units for the year, which was nearly a 15% increase from the same period in the previous year. Sales growth is particularly high amon

Turn 2 Answer:
 - Global EV Outlook predicts a significant increase in charging infrastructure by 2030, aiming for nearly 90 GW of public slow and almost 500 GW of fast installed cap